<a href="https://colab.research.google.com/github/sbylash/23_Shasia-Sasa_ML/blob/main/Uts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

Pada kuis ini, Anda diminta untuk melakukan klasifikasi citra wajah dengan menggunakan dataset [CISIA Webface](https://www.kaggle.com/datasets/ntl0601/casia-webface). Perhatian! Dataset ini cukup besar (2.8Gb) dari 500.000 gambar dan 10.575 subjek (label).

Spesifikasi pengerjaan UTS yang harus dipehuni adalah:

1. Pelajari tentang CISIA Webface!
2. Dikarenakan data gambar dari CISIA masih dalam 1 direktori besar, Anda perlu melakukan proses split antara data latih dan data uji. Anda dapat melakukan ini secara manual (langsung dari direktori) atau secara logikal dengan listing direktori. (10 poin)
3. Lakukan proses pra pengolahan data. Anda wajib dapat menjelaskan proses pra pengolahan data yang dilakukan. (20 poin)
4. Lakukan proses ekstraksi fitur. Fitur yang digunakan bebas. Anda wajib dapat menjelaskan fitur yang digunakan (30 poin)
5. Buat model NN dengan arsitektur yang kelompok Anda rancang sendiri. Model arsitektur bebas (jumlah layer, jumlah node, fungsi aktivasi). (30 poin)
6. Evaluasi performa model NN kelompok Anda dengan metrik akurasi, *precision*, *recall*, dan *F1-Score*. Jelaskan maksud dari metrik-metrik tersebut!


---

# Boilerplate

Berikut merupakan boilerplate code yang dapat Anda gunakan sebagai acuan dasar pengerjaan kuis. Anda diperkenankan untuk **menambah** ataupun **mengurangi** bagian boilerplate yang disediakan.

---

# Preparation

In [5]:
# Load required library
# Import Required Library

import os
import shutil
from sklearn.model_selection import train_test_split
!pip install kaggle

# Path ke dataset gambar
data_path = '/root/.cache/kagglehub/datasets/ntl0601/casia-webface/versions/1/casia-webface'

# Membuat directory untuk train dan test
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Mendapatkan daftar semua folder dalam dataset
folders = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]

# Exclude 'train' and 'test' folders from the list of folders to be split
folders = [f for f in folders if f not in ('train', 'test')]

# Check if 'folders' is empty and print a message if it is
if not folders:
    print(f"Warning: No subfolders found in '{data_path}'. Skipping train-test split.")
else:
    # Split data
    train_folders, test_folders = train_test_split(folders, test_size=0.2, random_state=42)

    # Fungsi untuk memindahkan file ke folder train atau test
    def move_files(folders, target_path):
        for folder in folders:
            full_folder_path = os.path.join(data_path, folder)
            target_folder_path = os.path.join(target_path, folder)
            shutil.move(full_folder_path, target_folder_path)

    # Memindahkan folder ke masing-masing directory
    move_files(train_folders, train_path)
    move_files(test_folders, test_path)

    print(f"Train data: {len(train_folders)} folders")
    print(f"Test data: {len(test_folders)} folders")

# Load Data

In [6]:
# Bagian ini dapat Anda gunakan untuk melakukan proses loading data dan juga proses split antara data latih dan data uji berdasarkan direktori gambar

def load_and_split_data(data_dir, test_size=0.2):
    filepaths, labels = [], []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                filepaths.append(os.path.join(label_dir, filename))
                labels.append(label)

    # Split data
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        filepaths, labels, test_size=test_size, stratify=labels, random_state=42
    )
    return train_paths, test_paths, train_labels, test_labels

# Preprocessing

In [11]:
'''
Pada bagian ini Anda diperbolehkan untuk melakukan proses pra pengolahan data (preprocessing) sesuai dengan kebutuhan. Pra pengolahan data dapat berupa:

1. Standardisasi nilai fitur ataupun label
2. Penyesuaian ukuran gambar
3. Perubahan colorspace gambar
4. dan lain-lain
'''

from sklearn.model_selection import train_test_split
import os

def load_and_split_data(data_dir):
    filepaths = []
    labels = []

    # Assume subdirectories in data_dir represent different classes
    for label_dir in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label_dir)
        if os.path.isdir(class_dir):
            for file in os.listdir(class_dir):
                filepaths.append(os.path.join(class_dir, file))
                labels.append(label_dir)  # or map label_dir to an integer if necessary

    # Split into train and test sets
    return train_test_split(filepaths, labels, test_size=0.2)

train_paths, test_paths, train_labels, test_labels = load_and_split_data(data_dir)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

# Features Extraction

In [ ]:
'''
Anda dapat melakukan proses ekstraksi fitur apapun sesuai dengan yang Anda inginkan
'''

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(set(train_labels)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Evaluation

In [ ]:
'''
Pada bagian ini, lakukan evaluasi terhadap data training dan data testing
'''
